In [ ]:
# FOR DKI,NODDI, AND MAP DIFFUSION FITTING
import os
import numpy as np
import matplotlib.pyplot as plt
import dipy.reconst.dki as dki
import dipy.reconst.dti as dti
from dipy.reconst import mapmri
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, get_sphere
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti,save_nifti
from dipy.segment.mask import median_otsu
from scipy.ndimage.filters import gaussian_filter
from dipy.viz import window, actor
from mpl_toolkits.axes_grid1 import make_axes_locatable
from dmipy.signal_models import cylinder_models, gaussian_models
from dmipy.distributions.distribute_models import SD1WatsonDistributed
from dmipy.core.acquisition_scheme import acquisition_scheme_from_bvalues
from dmipy.core.modeling_framework import MultiCompartmentModel

In [ ]:
workingdir="/media/peng/data/02_MriDataSet/99_demo/15_ukbiobank/wang/con001/dMRI/dMRI"
fraw="/media/peng/data/02_MriDataSet/99_demo/15_ukbiobank/wang/con001/dMRI/dMRI/data.nii.gz"
fbval="/media/peng/data/02_MriDataSet/99_demo/15_ukbiobank/wang/con001/dMRI/dMRI/bvals"
fbvec="/media/peng/data/02_MriDataSet/99_demo/15_ukbiobank/wang/con001/dMRI/dMRI/bvecs"
fmask="/media/peng/data/02_MriDataSet/99_demo/15_ukbiobank/wang/con001/dMRI/dMRI/nodif_brain_mask.nii.gz"
data, affine = load_nifti(fraw)
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab_dki = gradient_table(bvals, bvecs)
mask,mask_affine=load_nifti(fmask)

In [ ]:
dkimodel = dki.DiffusionKurtosisModel(gtab_dki)

In [ ]:
dkifit = dkimodel.fit(data, mask=mask)

In [ ]:
DKI_FA = dkifit.fa
DKI_MD = dkifit.md
DKI_AD = dkifit.ad
DKI_RD = dkifit.rd
DKI_MK = dkifit.mk(0, 3)
DKI_AK = dkifit.ak(0, 3)
DKI_RK = dkifit.rk(0, 3)
DKI_MKT = dkifit.mkt(0, 3)
DKI_KFA = dkifit.kfa

# MAP

In [ ]:
big_delta = 0.0365  # seconds
small_delta = 0.0157  # seconds
gtab_map = gradient_table(bvals=gtab_dki.bvals, bvecs=gtab_dki.bvecs,
                      big_delta=big_delta,
                      small_delta=small_delta)

In [ ]:
radial_order = 6
map_model_both_aniso = mapmri.MapmriModel(gtab_map, radial_order=radial_order,
                                          laplacian_regularization=True,
                                          laplacian_weighting=.2,
                                          positivity_constraint=False)

In [ ]:

mapfit_both_aniso = map_model_both_aniso.fit(data,mask)

In [ ]:
RTOP=mapfit_both_aniso.rtop()
RTAP=mapfit_both_aniso.rtap()
RTPP=mapfit_both_aniso.rtpp()
NORM=mapfit_both_aniso.norm_of_laplacian_signal()
MSD=mapfit_both_aniso.msd()
QIV=mapfit_both_aniso.qiv()

In [ ]:
map_model_both_ng = mapmri.MapmriModel(gtab_map, radial_order=radial_order,
                                        laplacian_regularization=True,
                                        laplacian_weighting=.2,
                                        positivity_constraint=False,
                                        bval_threshold=2000)           

In [ ]:
mapfit_both_ng = map_model_both_ng.fit(data,mask)

In [ ]:
NG=mapfit_both_ng.ng()
NG_PER=mapfit_both_ng.ng_perpendicular()
NG_PAR=mapfit_both_ng.ng_parallel()

# NODDI

In [ ]:
ball = gaussian_models.G1Ball()
stick = cylinder_models.C1Stick()
zeppelin = gaussian_models.G2Zeppelin()
watson_dispersed_bundle = SD1WatsonDistributed(models=[stick, zeppelin])
# watson_dispersed_bundle.parameter_names
watson_dispersed_bundle.set_tortuous_parameter('G2Zeppelin_1_lambda_perp','C1Stick_1_lambda_par','partial_volume_0')
watson_dispersed_bundle.set_equal_parameter('G2Zeppelin_1_lambda_par', 'C1Stick_1_lambda_par')
watson_dispersed_bundle.set_fixed_parameter('G2Zeppelin_1_lambda_par', 1.7e-9)
NODDI_mod = MultiCompartmentModel(models=[ball, watson_dispersed_bundle])
# NODDI_mod.parameter_names
NODDI_mod.set_fixed_parameter('G1Ball_1_lambda_iso', 3e-9)
acq_scheme = acquisition_scheme_from_bvalues(np.squeeze(bvals)*1e6, bvecs, small_delta, big_delta)

In [ ]:
NODDI_fit_result = NODDI_mod.fit(acq_scheme, data, mask=mask)
fitted_parameters = NODDI_fit_result.fitted_parameters

In [20]:
ISOVF=fitted_parameters['partial_volume_0']
ICVF=fitted_parameters['SD1WatsonDistributed_1_partial_volume_0']
OD=fitted_parameters['SD1WatsonDistributed_1_SD1Watson_1_odi']

In [18]:
os.chdir(workingdir)
#os.getcwd()

'/media/peng/data/02_MriDataSet/99_demo/15_ukbiobank/wang/con001/dMRI/dMRI'

In [21]:
#NODDI
save_nifti("dti_OD.nii.gz", OD,affine=affine)
save_nifti("dti_ICVF.nii.gz", ICVF,affine=affine)
save_nifti("dti_ISOVF.nii.gz", ISOVF,affine=affine)
# MAP
save_nifti("dti_RTOP.nii.gz", RTOP,affine=affine)
save_nifti("dti_RTAP.nii.gz", RTAP,affine=affine)
save_nifti("dti_RTPP.nii.gz", RTPP,affine=affine)
save_nifti("dti_NORM.nii.gz", NORM,affine=affine)
save_nifti("dti_MSD.nii.gz", MSD,affine=affine)
save_nifti("dti_QIV.nii.gz", QIV,affine=affine)
save_nifti("dti_NG.nii.gz", NG,affine=affine)
save_nifti("dti_NG_PER.nii.gz", NG_PER,affine=affine)
save_nifti("dti_NG_PAR.nii.gz", NG_PAR,affine=affine)
# dki
save_nifti("dti_DKI_FA.nii.gz", DKI_FA,affine=affine)
save_nifti("dti_DKI_MD.nii.gz", DKI_MD,affine=affine)
save_nifti("dti_DKI_AD.nii.gz", DKI_AD,affine=affine)
save_nifti("dti_DKI_RD.nii.gz", DKI_RD,affine=affine)
save_nifti("dti_DKI_MK.nii.gz", DKI_MK,affine=affine)
save_nifti("dti_DKI_AK.nii.gz", DKI_AK,affine=affine)
save_nifti("dti_DKI_RK.nii.gz", DKI_RK,affine=affine)
save_nifti("dti_DKI_MKT.nii.gz", DKI_MKT,affine=affine)
save_nifti("dti_DKI_KFA.nii.gz", DKI_KFA,affine=affine)